<a href="https://colab.research.google.com/github/emmaauerbach/Jensen/blob/main/Testing_Data_Cleaning_2021_11_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. Upload excel files into folder Testing_Files


In [1]:
!mkdir Out
!mkdir Testing_Files

In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
import os
import glob

In [4]:
#whether it's Ag or PCR
Ag_test = False


#path = os.getcwd()
path = "/content/Testing_Files/"
xlsx_files = glob.glob(os.path.join(path, "*.xlsx"))

counter = 0  
  
# loop over the list of xlsx files
for f in xlsx_files:
    #print file name at start
    file_name = f.split('/')[-1]
    file_name = file_name.split('.')[-2]
    print('Starting:', file_name)

    #determining the file type
    file_type = file_name[:2]
    print(file_type)
    if file_type == "PC":
      Ag_test = False
      print(Ag_test)
    elif file_type == "Ag":
      Ag_test = True
      print(Ag_test)
    else:
          raise Exception("Can't determine test type from the file name")  

    # read the xlsx file
    df = pd.read_excel(f)
    # Output with dates converted to YYYY-MM-DD
    df["Specimen_Coll_Dt"] = pd.to_datetime(df["Specimen_Coll_Dt"]).dt.strftime("%Y-%m-%d")
    df["Birth_Dt"] = pd.to_datetime(df["Birth_Dt"], dayfirst=True).dt.strftime("%Y-%m-%d")
    df["Lab_Report_Dt"] = pd.to_datetime(df["Lab_Report_Dt"]).dt.strftime("%Y-%m-%d")

    #change data types
    df["Testing_Lab_Accession_Number"] = df["Testing_Lab_Accession_Number"].astype(str)

    #add test type col
    if Ag_test:
      df['test_type'] = "Ag"
    else:
      df['test_type'] = "PCR" 


    #check if Ordered_Test_Desc is there
    if 'Ordered_Test_Desc' not in df:
      df.rename(columns = {'Resulted_Test_Desc':'Ordered_Test_Desc'}, inplace = True)

    #check if the address columns are there
    if 'Ordering_Facility_Address_One' not in df:
      df['Ordering_Facility_Address_One'] = "Address not provided in the ELR message"
      df['Ordering_Facility_City'] = np.nan
      df['Ordering_Facility_State'] = np.nan
      df['Ordering_Facility_Zip_Cd'] = np.nan

    #format zips 
    #remove zip+4s and just leave 5 digit zips
    df["Ordering_Facility_Zip_Cd"] = df["Ordering_Facility_Zip_Cd"].astype(str).str[:5]
    df['Zip_Code'] = df['Zip_Code'].astype(str).str[:5]
    #add source_file
    df['Source_File'] = file_name
    
    #reorder cols  
    columnsTitles = ['Zip_Code', 'Birth_Dt', 'Current_Sex_Cd', 'PATIENT_RACE_CALC', 'PATIENT_ETHNICITY', 'Testing_Lab_Accession_Number', 'Specimen_Coll_Dt', 'Lab_Report_Dt', 'Ordered_Test_Desc', 'Ordering_Facility_Name', 'Ordering_Provider_First_Name', 'Ordering_Provider_Last_Name', 'Perform_Facility_Name', 'Result_Category', 'Source_File', 'Ordering_Facility_Address_One', 'Ordering_Facility_City', 'Ordering_Facility_State', 'Ordering_Facility_Zip_Cd', 'test_type']
    df = df.reindex(columns=columnsTitles)
      
    #print at end
    print('Finished:', file_name)
    print(counter)
    counter += 1 

    df.to_csv("/content/Out/"+file_name+".csv", index=False )

Starting: Ag ELRs received 12-02-21
Ag
True
Finished: Ag ELRs received 12-02-21
0
Starting: PCR ELRs received 12-02-21
PC
False
Finished: PCR ELRs received 12-02-21
1


In [5]:
#save the CSVs into one zip to download 

!zip -r tests.zip /content/Out





  adding: content/Out/ (stored 0%)
  adding: content/Out/PCR ELRs received 12-02-21.csv (deflated 85%)
  adding: content/Out/Ag ELRs received 12-02-21.csv (deflated 88%)


In [7]:
#clean out the folders
!rm /content/Testing_Files/*
!rm /content/Out/*
